In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [2]:
# Load the CIFAR-100 dataset
(x_train, y_train), (x_test, y_test) = cifar100.load_data()

# Normalize the pixel values between 0 and 1
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Convert the labels to one-hot encoding
y_train = keras.utils.to_categorical(y_train, 100)
y_test = keras.utils.to_categorical(y_test, 100)


In [3]:
len(x_train)

50000

In [ ]:
#Method 1 

In [4]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import os
import glob

# Define the path to the ImageNet dataset
dataset_path = 'D:/data/imagenet'

# Get the list of class folders in the dataset
class_folders = glob.glob(dataset_path+'/*/')

In [11]:
import glob
import numpy as np
import gc
from tensorflow.keras.applications.resnet import ResNet50, preprocess_input
from PIL import Image
from tqdm import tqdm

def extract_features_from_directory(directory):
    image_list = []
    valid_extensions = ['*.jpg', '*.jpeg', '*.png', '*.gif']  # Add more valid extensions if needed

    # Load the ResNet50 model
    model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

    # Get the total number of files to process
    total_files = sum(len(glob.glob(os.path.join(directory, extension))) for extension in valid_extensions)

    # Iterate over the files and show progress using tqdm
    progress_bar = tqdm(total=total_files)
    for extension in valid_extensions:
        for image_path in glob.glob(os.path.join(directory, extension)):
            # Load and resize the image using PIL
            image = Image.open(image_path)
            image = image.resize((224, 224))

            # Convert image to array and normalize pixel values
            image = np.array(image) / 255.0

            # Add batch dimension to match ResNet input shape
            image = np.expand_dims(image, axis=0)

            # Preprocess the image
            image = preprocess_input(image)

            # Extract features using ResNet model
            features = model.predict(image)

            # Append the features to the list
            image_list.append(features)

            # Delete variables to clear memory
            del image_path, image
            gc.collect()

            # Update the progress bar
            progress_bar.update(1)

    # Close the progress bar
    progress_bar.close()

    # Delete the ResNet model
    del model
    gc.collect()

    return image_list


In [12]:
x = extract_features_from_directory(class_folders[0])

 31%|████████████████████████▋                                                      | 407/1300 [02:04<05:06,  2.91it/s]

ValueError: in user code:

    File "C:\Users\shaif\anaconda3\envs\tens\lib\site-packages\keras\engine\training.py", line 1621, in predict_function  *
        return step_function(self, iterator)
    File "C:\Users\shaif\anaconda3\envs\tens\lib\site-packages\keras\engine\training.py", line 1611, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\shaif\anaconda3\envs\tens\lib\site-packages\keras\engine\training.py", line 1604, in run_step  **
        outputs = model.predict_step(data)
    File "C:\Users\shaif\anaconda3\envs\tens\lib\site-packages\keras\engine\training.py", line 1572, in predict_step
        return self(x, training=False)
    File "C:\Users\shaif\anaconda3\envs\tens\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\shaif\anaconda3\envs\tens\lib\site-packages\keras\engine\input_spec.py", line 213, in assert_input_